In [1]:
%pip install numpy
%pip install matplotlib
%pip install seaborn
%pip install statsmodels

import numpy as np
import matplotlib.pyplot as plt

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# prices = np.loadtxt('prices.txt').T
# nInst, nt = prices.shape #number of instruments or stocks and number of time points
# # transposed the data so its 50x250

In [3]:
# np.random.seed(0)
# sample_inds = np.random.choice(nInst, size=3, replace=False)
# for idx in sample_inds:
#     plt.figure()
#     plt.plot(prices[idx])
#     plt.title(f"Instrument {idx} Price Over Time")
#     plt.xlabel("Day")
#     plt.ylabel("Price")

# # 2) Histogram of daily log returns across all instruments
# returns = np.diff(np.log(prices), axis=1)
# plt.figure()
# plt.hist(returns.flatten(), bins=50)
# plt.title("Histogram of Daily Log Returns")
# plt.xlabel("Log Return")
# plt.ylabel("Frequency")

# # 3) Correlation matrix heatmap of instrument returns
# corr = np.corrcoef(returns)
# plt.figure()
# plt.imshow(corr)
# plt.colorbar()
# plt.title("Correlation Matrix of Instrument Returns")
# plt.xlabel("Instrument")
# plt.ylabel("Instrument")

# plt.show()

In [4]:
import numpy as np
import pandas as pd

# Load your historical data
prices = pd.read_csv("prices.txt", delim_whitespace=True, header=None)

# # Transpose: we want rows = dates, columns = instruments
# price_df = prices.T

# Assign column names for clarity
prices.columns = [f"Stock{i+1}" for i in range(prices.shape[1])]

prices.tail()  # Optional: see the last few rows



C:\Users\user\AppData\Local\Temp\ipykernel_22788\2371662038.py:5: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  prices = pd.read_csv("prices.txt", delim_whitespace=True, header=None)


,Stock1,Stock2,Stock3,Stock4,Stock5,Stock6,Stock7,Stock8,Stock9,Stock10,...,Stock41,Stock42,Stock43,Stock44,Stock45,Stock46,Stock47,Stock48,Stock49,Stock50
745,36.49,51.34,45.11,29.37,39.73,28.69,17.89,47.71,40.70,58.56,...,75.13,46.74,39.54,32.21,75.55,30.62,85.74,16.13,81.57,31.53
746,36.30,50.95,44.66,29.32,39.85,28.59,17.84,47.31,40.32,58.05,...,75.06,47.19,39.32,32.92,75.97,30.87,85.88,15.91,80.50,31.65
747,36.58,50.80,44.25,29.17,40.01,28.97,17.76,47.01,39.97,57.61,...,74.97,47.52,39.58,33.31,76.05,30.83,86.15,15.81,80.93,31.83
748,36.43,51.19,44.02,29.11,40.01,28.54,17.69,45.68,39.86,59.06,...,75.36,48.15,39.94,32.95,75.38,30.81,86.42,15.75,80.80,31.78
749,36.24,50.64,43.43,29.01,40.03,28.70,17.69,46.00,39.78,59.52,...,74.90,48.46,40.10,32.91,75.01,30.94,86.86,15.74,80.50,31.59


In [ ]:
# correlation matrix of the prices
corr_matrix = prices.corr().abs()
corr_matrix


In [6]:
#heat matrix of the correlation matrix

''' Uncomment the following lines to visualize the correlation matrix in a heat map '''


# import seaborn as sns
# import matplotlib.pyplot as plt
# from matplotlib.pyplot import figure
# figure(figsize=(12, 10), dpi=80)
# sns.heatmap(corr_matrix, annot=False)
# plt.title("Correlation Matrix Heatmap of Stock Prices")
# plt.show()


' Uncomment the following lines to visualize the correlation matrix in a heat map '

In [7]:
corr_unstacked = corr_matrix.unstack()

# Filter out self-correlations and group by pairs and remove the duplicate pairs - CHATGPT did this for me
corr_unstacked = corr_unstacked[corr_unstacked.index.get_level_values(0) != corr_unstacked.index.get_level_values(1)]
corr_unstacked = corr_unstacked.groupby([frozenset(pair) for pair in corr_unstacked.index]).mean()

#we want to find the top pairs with highest correlation that is atleast above 0.8 (**chosen threshold can be adjusted**)
correlation_threshold = 0.8
corr_unstacked = corr_unstacked[corr_unstacked > correlation_threshold]
# Sort the pairs by correlation value
top_pairs = corr_unstacked.sort_values(ascending=False)

print(top_pairs)

(Stock7, Stock3)      0.938054
(Stock5, Stock21)     0.926042
(Stock21, Stock3)     0.918227
(Stock5, Stock7)      0.917858
(Stock50, Stock2)     0.906795
                        ...   
(Stock7, Stock44)     0.803467
(Stock23, Stock36)    0.801468
(Stock49, Stock50)    0.801410
(Stock24, Stock48)    0.800898
(Stock32, Stock28)    0.800677
Length: 69, dtype: float64


In [8]:
#graph of the two pairs with the highest correlation from above
def plot_pair(stock1, stock2, prices):
    plt.figure(figsize=(10, 5), dpi=80)
    plt.plot(prices[stock1], label=stock1)
    plt.plot(prices[stock2], label=stock2)
    plt.title(f'{stock1} vs {stock2} – Price Comparison')
    plt.xlabel('Days')
    plt.ylabel('Price')
    plt.legend()
    plt.grid(True)
    plt.show()

#plot the spread between the two pairs with the highest correlation
def plot_spread(stock1, stock2, prices):
    spread = prices[stock1] - prices[stock2]
    plt.figure(figsize=(10, 5), dpi=80)
    plt.plot(spread, label=f'Spread: {stock1} - {stock2}', color='orange')
    plt.title(f'Spread between {stock1} and {stock2}')
    plt.xlabel('Days')
    plt.ylabel('Spread Price')
    plt.legend()
    plt.grid(True)
    plt.show()

#check out the cointegration value: Null hyp. = no cointegration
#pretty irrelevant now as the function below checks cointegration for all pairs that have a high correlation
import statsmodels.tsa.stattools as ts
def check_cointegration(stock1, stock2, prices):
    result = ts.coint(prices[stock1], prices[stock2])
    print(f'Cointegration test between {stock1} and {stock2}:')
    # print(f'  Test Statistic: {result[0]}')
    print(f'  p-value: {result[1]}')
    # print(f'  Critical Values: {result[2]}')
    print(f'WANT p-value < 0.05 for cointegration (shows that cointegration exists)')

In [9]:
#loop through the top pairs and check cointegration

from statsmodels.tsa.stattools import coint

def check_cointegration_all(top_pairs, prices, pval_threshold=0.05):
    cointegrated_pairs = []
    for (stock1, stock2), corr_value in top_pairs.items():
        series1 = prices[stock1]
        series2 = prices[stock2]
        score, p_value, _ = coint(series1, series2)
        if p_value < pval_threshold:
            cointegrated_pairs.append((stock1, stock2, p_value))
    return pd.DataFrame(cointegrated_pairs)

In [10]:
# plot_pair('Stock3', 'Stock5', prices)
# plot_spread('Stock3', 'Stock5', prices)
# check_cointegration('Stock3', 'Stock5', prices)

print(f'Wanted a p-value < 0.05 for cointegration (shows that cointegration exists) this resulted in the following cointegrated pairs that had a > 0.8 correlation:')

# Get the top pairs and check cointegration
cointegrated_pairs = check_cointegration_all(top_pairs, prices)
print("Cointegrated Pairs:")
print(cointegrated_pairs)


Wanted a p-value < 0.05 for cointegration (shows that cointegration exists) this resulted in the following cointegrated pairs that had a > 0.8 correlation:
Cointegrated Pairs:
          0        1         2
0    Stock7   Stock3  0.008298
1   Stock21   Stock3  0.043106
2   Stock50   Stock2  0.003220
3   Stock23  Stock21  0.014386
4   Stock24  Stock12  0.013460
5   Stock23   Stock3  0.031725
6   Stock34   Stock2  0.011937
7   Stock30  Stock43  0.019083
8   Stock23   Stock7  0.017421
9   Stock30  Stock17  0.028154
10  Stock50  Stock34  0.000463
11  Stock23  Stock30  0.029159
12  Stock32  Stock10  0.016146
13  Stock50  Stock26  0.011013
14  Stock32  Stock15  0.037466
15   Stock8  Stock14  0.025605
16  Stock49  Stock50  0.000028
